In [37]:
# importing necessary libraries
import pandas as pd
import yfinance as yf
import requests


In [38]:
df = pd.read_csv(r'raw_data/sp_500_historical_components.csv')

# Data cleaning
In this portion of the code we will 
1) Get the list of stocks that ever existed from sp_500_historical_components.csv from https://github.com/hanshof/sp500_constituents/blob/main/sp_500_historical_components.csv
2) Use yfinance library to get historical data (2013-2020) OHLCV, PE ratio, PB ratio from the lists of stocks, indicate their presence in the stock data at any point of time on a daily level
3) Only use stocks that are listed at the start of the month to the end of the month, e.g: if a stock enters the stock market on 4 Jan 2016, we will only use the data starting from 1 feb onwards
4) Filter out stocks that newly entered the stock market before 2016, this is to ensure that we have sufficient training data for each stock

In [ ]:
# Firstly, we will get the list of all stocks that ever existed in the S&P 500 from 2013 to 2020

# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Ensure 'tickers' is string
df['tickers'] = df['tickers'].astype(str)

# Filter date range
df = df[(df['date'] >= '2012-01-01') & (df['date'] < '2021-01-08')]

In [40]:
# Function to parse tickers from CSV string
def parse_ticker_list(s):
    if pd.isna(s): 
        return []
    s = s.strip()
    if s.startswith('"') and s.endswith('"'):
        s = s[1:-1]
    items = [t.strip() for t in s.split(',') if t.strip() != '']
    return items

# Apply parsing
df['ticker_list'] = df['tickers'].apply(parse_ticker_list)

# Keep only relevant columns
df = df[['date', 'ticker_list']]

In [41]:
def replace_fb_with_meta(ticker_list):
    return ['META' if ticker == 'FB' else ticker for ticker in ticker_list]

df['ticker_list'] = df['ticker_list'].apply(replace_fb_with_meta)

In [42]:
# Extract first date of each month
df['year_month'] = df['date'].dt.to_period('M')  # e.g., 2020-01
month_starts = df.groupby('year_month')['date'].min().reset_index()
month_starts.columns = ['year_month', 'start_date']

print(month_starts)
# Loop over each month and get tickers that were present throughout the month
# If the stock was missing on any date in that month, it is excluded from the list of tickers for that month

results = []

for i in range(len(month_starts) - 1):
    start = month_starts.loc[i, 'start_date']
    end = month_starts.loc[i + 1, 'start_date']

    # Filter rows from start to just before next month’s start
    month_df = df[(df['date'] >= start) & (df['date'] < end)]

    all_tickers = set().union(*month_df['ticker_list'])

    # Get intersection of ticker_lists in this period
    if not month_df.empty:
        surviving = set(month_df.iloc[0]['ticker_list'])
        for tickers in month_df['ticker_list']:
            surviving &= set(tickers)

        not_survived = all_tickers - surviving

        results.append({
            'date': start,
            'ticker_list': sorted(list(surviving)),
            'not_survived': sorted(list(not_survived)),
            'all tickers': len(all_tickers)
        })

   year_month start_date
0     2013-01 2013-01-02
1     2013-02 2013-02-04
2     2013-03 2013-03-11
3     2013-04 2013-04-01
4     2013-05 2013-05-01
..        ...        ...
87    2020-09 2020-09-18
88    2020-10 2020-10-07
89    2020-11 2020-11-17
90    2020-12 2020-12-21
91    2021-01 2021-01-07

[92 rows x 2 columns]


In [43]:
# Create final tickers DataFrame
survivors_df = pd.DataFrame(results)
survivors_df['num_survivors'] = survivors_df['ticker_list'].apply(len)

# Reindex to ensure all months are present, forward-fill missing months
survivors_df['month'] = survivors_df['date'].dt.to_period('M').dt.to_timestamp()
survivors_df.set_index('month', inplace=True)
full_month_range = pd.date_range('2013-01-01', '2020-12-01', freq='MS')
survivors_df = survivors_df.reindex(full_month_range)
survivors_df.ffill(inplace=True)
survivors_df.reset_index(inplace=True)
survivors_df.rename(columns={'index': 'month'}, inplace=True)

survivors_df

,month,date,ticker_list,not_survived,all tickers,num_survivors
0,2013-01-01,2013-01-02,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...",[],458.0,458.0
1,2013-02-01,2013-02-04,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...","[BIG, PVH]",459.0,457.0
2,2013-03-01,2013-03-11,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...",[],458.0,458.0
3,2013-04-01,2013-04-01,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...",[],458.0,458.0
4,2013-05-01,2013-05-01,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...","[DF, KSU, MAC]",460.0,457.0
...,...,...,...,...,...,...
91,2020-08-01,2020-06-22,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",[],501.0,501.0
92,2020-09-01,2020-09-18,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...","[COTY, CTLT, ETSY, HRB, KSS, TER]",504.0,498.0
93,2020-10-01,2020-10-07,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...","[NBL, VNT]",502.0,500.0
94,2020-11-01,2020-11-17,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",[],501.0,501.0


In [44]:
# Explode the ticker_list so each row has one ticker
exploded = survivors_df.explode('ticker_list')

# Assign presence flag
exploded['value'] = 1

# Pivot the table
pivot_df = exploded.pivot_table(
    index='date',        # Each row is a month
    columns='ticker_list',
    values='value',
    fill_value=0         # If the ticker wasn't present, put 0
)

#sort columns (tickers)
pivot_df = pivot_df.sort_index(axis=1)
pivot_df = pivot_df.sort_index(axis=0)


pivot_df

ticker_list,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
2013-02-04,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
2013-03-11,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
2013-04-01,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
2013-05-01,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-22,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2020-09-18,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2020-10-07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [45]:
summary_data = []

for ticker in pivot_df.columns:
    series = pivot_df[ticker]
    active_months = series[series == 1]

    if not active_months.empty:
        summary_data.append({
            'ticker': ticker,
            'first_seen': active_months.index.min(),
            'last_seen': active_months.index.max(),
            'months_active': active_months.count()
        })

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('ticker').reset_index(drop=True)


summary_df

,ticker,first_seen,last_seen,months_active
0,A,2013-01-02,2020-12-21,91
1,AAL,2015-04-07,2020-12-21,64
2,AAP,2015-08-04,2020-12-21,60
3,AAPL,2013-01-02,2020-12-21,91
4,ABBV,2013-01-02,2020-12-21,91
...,...,...,...,...
635,YUM,2013-01-02,2020-12-21,91
636,ZBH,2013-01-02,2020-12-21,91
637,ZBRA,2020-01-28,2020-12-21,9
638,ZION,2013-01-02,2020-12-21,91


In [46]:
# Ensure dates are datetime 
summary_df['first_seen'] = pd.to_datetime(summary_df['first_seen'])
summary_df['last_seen'] = pd.to_datetime(summary_df['last_seen'])

# Function to get first and last day of the month
def get_month_range(start, end):
    start_of_month = start.to_period('M').to_timestamp()       # first day of month
    end_of_month = end.to_period('M').to_timestamp('M')        # last day of month
    return start_of_month, end_of_month

# Function to fetch stock data from Yahoo Finance
def get_stock_data(ticker, start_date, end_date):
    try:
        stock = yf.Ticker(ticker)
        stock_data = stock.history(start=start_date, end=end_date + pd.Timedelta(days=1))  # Add 1 day to include end date
        stock_data['ticker'] = ticker
        return stock_data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

# Loop through summary_df and collect stock data
stock_data_list = []
success_count = 0  # Counter for successful tickers

for idx, row in summary_df.iterrows():
    ticker = row['ticker']
    first_seen = row['first_seen']
    last_seen = row['last_seen']
    
    # Adjust to first and last day of the month
    start_date, end_date = get_month_range(first_seen, last_seen)
    
    stock_data = get_stock_data(ticker, start_date, end_date)
    
    if not stock_data.empty:
        stock_data_list.append(stock_data)
        success_count += 1  # Increment if data fetched successfully

# Combine all stock data
all_stock_data = pd.concat(stock_data_list)
all_stock_data.reset_index(inplace=True)

# Print count of tickers successfully fetched
print(f"Number of tickers successfully fetched: {success_count}")

$ABC: possibly delisted; no timezone found
$ABMD: possibly delisted; no timezone found
$ADS: possibly delisted; no timezone found
$ALXN: possibly delisted; no timezone found
$ANSS: possibly delisted; no timezone found
$ANTM: possibly delisted; no timezone found
$APC: possibly delisted; no timezone found
$APOL: possibly delisted; no price data found  (1d 2013-01-01 00:00:00 -> 2013-07-01 00:00:00)
$ARG: possibly delisted; no price data found  (1d 2013-01-01 00:00:00 -> 2016-05-01 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1357016400, endDate = 1462075200")
$ATVI: possibly delisted; no timezone found
$AVP: possibly delisted; no timezone found
$BCR: possibly delisted; no price data found  (1d 2013-01-01 00:00:00 -> 2017-12-01 00:00:00)
$BIG: possibly delisted; no timezone found
$BLL: possibly delisted; no timezone found
$BRCM: possibly delisted; no price data found  (1d 2013-01-01 00:00:00 -> 2016-02-01 00:00:00)
$BTUUQ: possibly delisted; no price data found  (1d 2013-0

Number of tickers successfully fetched: 515


In [47]:
all_stock_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,Capital Gains
0,2013-01-02 00:00:00-05:00,27.067739,27.067739,26.413031,26.881598,8790205,0.0,0.0,A,NaN
1,2013-01-03 00:00:00-05:00,26.920117,27.048492,26.689042,26.977886,5751791,0.0,0.0,A,NaN
2,2013-01-04 00:00:00-05:00,27.048492,27.568409,26.868767,27.510639,6432897,0.0,0.0,A,NaN
3,2013-01-07 00:00:00-05:00,27.343752,27.472127,27.202540,27.311659,3589505,0.0,0.0,A,NaN
4,2013-01-08 00:00:00-05:00,27.260305,27.459286,27.022812,27.093418,3896925,0.0,0.0,A,NaN
...,...,...,...,...,...,...,...,...,...,...
831335,2020-12-24 00:00:00-05:00,153.424249,155.295040,153.357100,154.191757,417400,0.0,0.0,ZTS,NaN
831336,2020-12-28 00:00:00-05:00,154.882536,156.216075,153.798430,155.793945,1522400,0.0,0.0,ZTS,NaN
831337,2020-12-29 00:00:00-05:00,156.580618,158.393848,155.803507,156.494263,1188400,0.0,0.0,ZTS,NaN
831338,2020-12-30 00:00:00-05:00,156.868426,158.106036,156.532651,157.597565,1009000,0.0,0.0,ZTS,NaN


In [48]:
SP500_all_stock_data = all_stock_data.drop(columns=['Stock Splits']) # dropping since we already account for the price adjustments in 'Adj Close'


#Daily returns
SP500_all_stock_data['daily_return'] = (
    SP500_all_stock_data.groupby('ticker')['Close']
    .pct_change()
)

#Monthly returns
monthly_returns = (
    SP500_all_stock_data
    .set_index('Date')
    .groupby('ticker')['Close']
    .resample('ME')
    .ffill()             # forward-fill missing days within month
    .pct_change()        # monthly percentage change
    .reset_index(name='monthly_return')
)

SP500_all_stock_data = SP500_all_stock_data.merge(
    monthly_returns[['Date', 'ticker', 'monthly_return']],
    on=['Date', 'ticker'],
    how='left'
)

#Check index membership, if tikcer is in ticker_list for that month
SP500_all_stock_data['Date'] = SP500_all_stock_data['Date'].dt.tz_localize(None)
SP500_all_stock_data['month'] = SP500_all_stock_data['Date'].dt.to_period('M').dt.to_timestamp()


SP500_all_stock_data = SP500_all_stock_data.merge(
    survivors_df[['month', 'ticker_list']],
    on='month',
    how='left'
)

SP500_all_stock_data['membership_index'] = SP500_all_stock_data.apply(
    lambda row: row['ticker'] in row['ticker_list'],
    axis=1
)

SP500_all_stock_data

,Date,Open,High,Low,Close,Volume,Dividends,ticker,Capital Gains,daily_return,monthly_return,month,ticker_list,membership_index
0,2013-01-02,27.067739,27.067739,26.413031,26.881598,8790205,0.0,A,NaN,NaN,NaN,2013-01-01,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...",True
1,2013-01-03,26.920117,27.048492,26.689042,26.977886,5751791,0.0,A,NaN,0.003582,NaN,2013-01-01,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...",True
2,2013-01-04,27.048492,27.568409,26.868767,27.510639,6432897,0.0,A,NaN,0.019748,NaN,2013-01-01,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...",True
3,2013-01-07,27.343752,27.472127,27.202540,27.311659,3589505,0.0,A,NaN,-0.007233,NaN,2013-01-01,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...",True
4,2013-01-08,27.260305,27.459286,27.022812,27.093418,3896925,0.0,A,NaN,-0.007991,NaN,2013-01-01,"[A, AAPL, ABBV, ABC, ABT, ACN, ADBE, ADI, ADM,...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831335,2020-12-24,153.424249,155.295040,153.357100,154.191757,417400,0.0,ZTS,NaN,0.005443,NaN,2020-12-01,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",True
831336,2020-12-28,154.882536,156.216075,153.798430,155.793945,1522400,0.0,ZTS,NaN,0.010391,NaN,2020-12-01,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",True
831337,2020-12-29,156.580618,158.393848,155.803507,156.494263,1188400,0.0,ZTS,NaN,0.004495,NaN,2020-12-01,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",True
831338,2020-12-30,156.868426,158.106036,156.532651,157.597565,1009000,0.0,ZTS,NaN,0.007050,NaN,2020-12-01,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",True


In [49]:
SP500_all_stock_data = SP500_all_stock_data.drop(columns=['ticker_list','month','Capital Gains'])

SP500_all_stock_data

,Date,Open,High,Low,Close,Volume,Dividends,ticker,daily_return,monthly_return,membership_index
0,2013-01-02,27.067739,27.067739,26.413031,26.881598,8790205,0.0,A,NaN,NaN,True
1,2013-01-03,26.920117,27.048492,26.689042,26.977886,5751791,0.0,A,0.003582,NaN,True
2,2013-01-04,27.048492,27.568409,26.868767,27.510639,6432897,0.0,A,0.019748,NaN,True
3,2013-01-07,27.343752,27.472127,27.202540,27.311659,3589505,0.0,A,-0.007233,NaN,True
4,2013-01-08,27.260305,27.459286,27.022812,27.093418,3896925,0.0,A,-0.007991,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
831335,2020-12-24,153.424249,155.295040,153.357100,154.191757,417400,0.0,ZTS,0.005443,NaN,True
831336,2020-12-28,154.882536,156.216075,153.798430,155.793945,1522400,0.0,ZTS,0.010391,NaN,True
831337,2020-12-29,156.580618,158.393848,155.803507,156.494263,1188400,0.0,ZTS,0.004495,NaN,True
831338,2020-12-30,156.868426,158.106036,156.532651,157.597565,1009000,0.0,ZTS,0.007050,NaN,True


### Creating new features
#### Moving averages
https://www.investopedia.com/ask/answers/122414/what-are-most-common-periods-used-creating-moving-average-ma-lines.asp we will use short:20 days, medium: 50 days, long: 100 days moving averages
1) Simple moving average SMA: SMA_20, SMA_50, SMA_100
2) Exponential Moving Average EMA https://www.investopedia.com/terms/e/ema.asp#toc-formula-for-exponential-moving-average-ema: EMA_20, EMA_50, EMA_100
3) Relative Strength Index 14 days https://www.investopedia.com/terms/r/rsi.asp : RSI
4) Moving Average Convergence/Divergence indicator: https://www.investopedia.com/terms/m/macd.asp: MACD_26, MACD_12, MACD_9 

In [50]:
def sma(series: pd.Series, window: int) -> pd.Series:
    """Simple Moving Average."""
    return series.rolling(window, min_periods=window).mean()


In [51]:
def ema(series: pd.Series, span: int) -> pd.Series:
    """Exponential Moving Average."""
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

In [52]:
def rsi_wilder(series: pd.Series, period: int = 14) -> pd.Series:
    """Wilder's RSI (default 14)."""
    delta = series.diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False, min_periods=period).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False, min_periods=period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

In [53]:
def macd(series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    """
    MACD parts: returns DataFrame with MACD_Line, MACD_Signal, MACD_Hist.
    """
    ema_fast = ema(series, fast)
    ema_slow = ema(series, slow)
    macd_line = ema_fast - ema_slow
    macd_signal = macd_line.ewm(span=signal, adjust=False, min_periods=signal).mean()
    macd_hist = macd_line - macd_signal
    return pd.DataFrame(
        {"MACD_Line": macd_line, "MACD_Signal": macd_signal, "MACD_Hist": macd_hist},
        index=series.index
    )

In [54]:
df = SP500_all_stock_data.copy()

# Ensure types/order; compute per-ticker
df['Date'] = pd.to_datetime(df['Date'])
df = df.drop_duplicates(subset=['ticker','Date']).sort_values(['ticker','Date'])

# Prefer adjusted close if available
price_col = 'Adj Close' if 'Adj Close' in df.columns else 'Close'

g = df.groupby('ticker', group_keys=False)

# SMA 20/50/100
for w in [20, 50, 100]:
    df[f'SMA_{w}'] = g[price_col].transform(lambda s, w=w: sma(s, w))

# EMA 20/50/100
for w in [20, 50, 100]:
    df[f'EMA_{w}'] = g[price_col].transform(lambda s, w=w: ema(s, w))

# RSI 14
df['RSI'] = g[price_col].transform(rsi_wilder)

# MACD (12,26,9)
macd_df = g[price_col].apply(macd)
df = df.join(macd_df)

# Hand back to your master frame
SP500_all_stock_data_Final = df

In [55]:
SP500_all_stock_data_Final

,Date,Open,High,Low,Close,Volume,Dividends,ticker,daily_return,monthly_return,...,SMA_20,SMA_50,SMA_100,EMA_20,EMA_50,EMA_100,RSI,MACD_Line,MACD_Signal,MACD_Hist
0,2013-01-02,27.067739,27.067739,26.413031,26.881598,8790205,0.0,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-03,26.920117,27.048492,26.689042,26.977886,5751791,0.0,A,0.003582,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-04,27.048492,27.568409,26.868767,27.510639,6432897,0.0,A,0.019748,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-07,27.343752,27.472127,27.202540,27.311659,3589505,0.0,A,-0.007233,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-08,27.260305,27.459286,27.022812,27.093418,3896925,0.0,A,-0.007991,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831335,2020-12-24,153.424249,155.295040,153.357100,154.191757,417400,0.0,ZTS,0.005443,NaN,...,153.626685,155.672890,154.709184,154.477094,154.663481,151.699309,48.711779,-0.298373,-0.514777,0.216404
831336,2020-12-28,154.882536,156.216075,153.798430,155.793945,1522400,0.0,ZTS,0.010391,NaN,...,153.670817,155.665287,154.755878,154.602508,154.707813,151.780391,52.813385,-0.160576,-0.443937,0.283361
831337,2020-12-29,156.580618,158.393848,155.803507,156.494263,1188400,0.0,ZTS,0.004495,NaN,...,153.802251,155.660958,154.774986,154.782675,154.777870,151.873735,54.525262,0.005081,-0.354133,0.359214
831338,2020-12-30,156.868426,158.106036,156.532651,157.597565,1009000,0.0,ZTS,0.007050,NaN,...,153.941840,155.746919,154.828602,155.050760,154.888446,151.987078,57.162001,0.222824,-0.238742,0.461566
